In [1]:
from goesaws import *
import datetime
import pandas as pd
import sys

date = str(datetime.date.today())

bucket_name = 'noaa-goes16'
product_name = 'ABI-L2-MCMIPM'
lightning_mapper = 'GLM-L2-LCFA'
yr = 2022
day_of_year =141
hr = 18
minutes = 30

In [2]:
abiprefix = gen_prefix(product=product_name,year = yr, day=day_of_year, hour = hr)
abifiles = gen_fn(bucket=bucket_name,prefix=abiprefix)
abidatasets = [gen_data(key=abifiles[i], bucket = bucket_name) for i in range(0,minutes)]
abiDS = xr.concat(abidatasets,dim = 't')

In [3]:
skyprefix = gen_prefix(product = "ABI-L2-ACMM", year = yr, day = day_of_year, hour = hr)
skyfiles = gen_fn(bucket = bucket_name, prefix=skyprefix)
skydatasets = [gen_data(key=skyfiles[i], bucket = bucket_name) for i in range(0,minutes)]
skyDS = xr.concat(skydatasets,dim = 't')

In [4]:
dataset2_resampled = skyDS.interp(
    x=abiDS["x"].data,
    y=abiDS["y"].data,
    t = abiDS["t"].data,
    method='linear',
    kwargs={'fill_value': "extrapolate"}
)

In [5]:
nfile = xr.merge([abiDS,dataset2_resampled])
nfile = nfile.coarsen(x=4,y=4).mean()
nfile = calc_latlon(nfile)
#nfile

In [6]:
df = nfile.to_dataframe()

In [7]:
features = ["CMI_C01", "CMI_C02","CMI_C03","CMI_C04","CMI_C05","CMI_C06","CMI_C07","CMI_C08","CMI_C09","CMI_C10","CMI_C11","CMI_C12","CMI_C13","CMI_C14","CMI_C15","CMI_C16","ACM", "BCM", "Cloud_Probabilities","lat","lon"]
ndf = df[features].copy()
ndf["time"] = ndf.index.get_level_values('t')
ndf.drop_duplicates(inplace=True)


In [8]:
### We want to get the lightning data from the GLM-L2-LCFA product
### We will use the same time period as the ABI data

glmprefix = gen_prefix(product = "GLM-L2-LCFA", year = yr, day = day_of_year, hour = hr)
glmfiles = gen_fn(bucket = bucket_name, prefix=glmprefix)
#There should be 180 GLM files per hour (one every 20 seconds)
glmdatasets = [gen_data(key=glmfiles[i], bucket = bucket_name) for i in range(0,minutes*3)]
latitudes = [np.array(glmdatasets[i]["event_lat"].data) for i in range(0,minutes*3)]
lats = np.concatenate(latitudes)
longitudes = [np.array(glmdatasets[i]["event_lon"].data) for i in range(0,minutes*3)]
lons= np.concatenate(longitudes)
times = [np.array(glmdatasets[i]["event_time_offset"].data) for i in range(0,minutes*3)]
times = np.concatenate(times)

strikes = list(zip(lats,lons))

In [9]:
from scipy.spatial import cKDTree
import pandas as pd

ndf = ndf.reset_index(drop=True)  # reset the index of the ndf dataframe
#ndf['time'] = ndf['time'].astype(np.int64)  # convert the time column to datetime objects
tree = cKDTree(ndf[['lat', 'lon']].values)
distances, indices = tree.query(strikes)

# filter out the indices that are not present in the ndf dataframe
valid_indices = indices[indices < len(ndf)]

lightning_df = pd.DataFrame({
    'strike_lat': [strike[0] for strike in strikes],
    'strike_lon': [strike[1] for strike in strikes],
    'time': times, 
    'nearest_lat': ndf.loc[valid_indices, 'lat'].values,
    'nearest_lon': ndf.loc[valid_indices, 'lon'].values,
    'distance': distances[indices < len(ndf)]
})


lightning_df["lightning"] = 0
lightning_df.loc[distances < 5, 'lightning'] = 1
lightning_df["Coordinates"] = list(zip(lightning_df["nearest_lat"],lightning_df["nearest_lon"]))



In [10]:
ndf['time_int'] = ndf['time'].astype(np.int64)

tree = cKDTree(ndf[['time_int']].values.reshape(-1, 1))

time_query = lightning_df['time'].astype(np.int64).values

distances, indices = tree.query(time_query.reshape(-1, 1))

lightning_df['nearest_time'] = ndf.loc[indices, 'time'].values



In [11]:
features = ["CMI_C01", "CMI_C02","CMI_C03","CMI_C04","CMI_C05","CMI_C06","CMI_C07","CMI_C08","CMI_C09","CMI_C10","CMI_C11","CMI_C12","CMI_C13","CMI_C14","CMI_C15","CMI_C16","ACM", "BCM", "Cloud_Probabilities","lat","lon","Coordinates","time","Lightning"]
ndf["Coordinates"] = list(zip(ndf["lat"],ndf["lon"]))





In [12]:
strike_df = lightning_df[lightning_df["lightning"] == 1]

# strike_df = strike_df.sort_values(by=['time'])

# ### Merge strike_df with ndf based on coordinate and nearest time
# def find_nearest_time(x):
#     return ndf.iloc[(np.abs(ndf['time'] - x['time'])).argmin()]['time']

# # Add a second column to df1 with the nearest time from df2
# strike_df['nearest_time'] = strike_df.apply(lambda x: find_nearest_time(x), axis=1)

# strike_df





In [13]:
## join Strike_df and ndf on nearest time and coordinates, filling lightning with 0 if na 

merged_df = strike_df.merge(ndf, left_on=['nearest_time','Coordinates'], right_on=['time','Coordinates'], how='left')
#merged_df = merged_df.fillna(0)
merged_df.drop_duplicates(inplace=True)
we_want = merged_df.groupby(['Coordinates','nearest_time']).count()["lightning"].reset_index()



In [14]:
## merge two dataframes on nearest time and coordinates, filling lightning with 0 if na

final_df = we_want.merge(ndf, left_on = ['nearest_time','Coordinates'], right_on = ['time','Coordinates'], how = 'right')
final_df = final_df.fillna(0)
final_df["Lightning"] = final_df["lightning"].apply(lambda x: 1 if x > 0 else 0)
final_df 

,Coordinates,nearest_time,lightning,CMI_C01,CMI_C02,CMI_C03,CMI_C04,CMI_C05,CMI_C06,CMI_C07,...,CMI_C15,CMI_C16,ACM,BCM,Cloud_Probabilities,lat,lon,time,time_int,Lightning
0,"(40.13103323474366, -93.38155072424266)",0,0.0,0.481706,0.431289,0.579424,0.002143,0.478710,0.376825,302.558289,...,274.171387,263.115356,2.156176,0.937424,0.762866,40.131033,-93.381551,2022-05-21 18:00:31.272519936,1653156031272519936,0
1,"(40.12698592712501, -93.2741436595977)",0,0.0,0.365158,0.305932,0.495337,0.002024,0.376627,0.285793,300.493988,...,274.726074,263.449341,1.843675,0.843675,0.530524,40.126986,-93.274144,2022-05-21 18:00:31.272519936,1653156031272519936,0
2,"(40.122965705716815, -93.16685328500078)",0,0.0,0.318968,0.255297,0.471785,0.003135,0.333135,0.245873,299.167969,...,274.573425,263.160126,1.874843,0.812655,0.615035,40.122966,-93.166853,2022-05-21 18:00:31.272519936,1653156031272519936,0
3,"(40.11897247748837, -93.05967862352706)",0,0.0,0.310932,0.245516,0.473551,0.006746,0.323313,0.240774,298.233246,...,272.764282,261.356201,2.905629,1.000000,0.980994,40.118972,-93.059679,2022-05-21 18:00:31.272519936,1653156031272519936,0
4,"(40.115006150366405, -92.95261870531907)",0,0.0,0.373254,0.310456,0.511190,0.020992,0.365952,0.287559,295.210480,...,265.780762,255.517563,3.000000,1.000000,0.999029,40.115006,-92.952619,2022-05-21 18:00:31.272519936,1653156031272519936,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468745,"(27.80295961693488, -80.17422458142033)",0,0.0,0.218750,0.148174,0.146369,0.056667,0.089246,0.086190,283.222595,...,259.579010,249.844177,3.000000,1.000000,0.999214,27.802960,-80.174225,2022-05-21 18:29:28.674865024,1653157768674865024,0
468746,"(27.802322483769377, -80.09039916473711)",0,0.0,0.182659,0.111151,0.103472,0.039286,0.062837,0.056984,287.639252,...,268.029175,256.109680,3.000000,1.000000,0.999220,27.802322,-80.090399,2022-05-21 18:29:28.674865024,1653157768674865024,0
468747,"(27.801695861529403, -80.0065919270383)",0,0.0,0.151210,0.077242,0.065258,0.017817,0.040496,0.032381,294.408356,...,279.100037,264.623230,3.000000,1.000000,0.999315,27.801696,-80.006592,2022-05-21 18:29:28.674865024,1653157768674865024,0
468748,"(27.801079742704182, -79.92280256024858)",0,0.0,0.166885,0.094841,0.087282,0.012222,0.067837,0.056052,297.324707,...,280.246613,265.886658,3.000000,1.000000,0.999431,27.801080,-79.922803,2022-05-21 18:29:28.674865024,1653157768674865024,0


Right now, all of the lightning strikes might not necessarily be close to the assigned value. Also we have to deal with the time component. But this is much better than before. 

In [15]:
final_df.to_csv("/Users/robbiefeldstein/Documents/Programming/Research/Datasets/May_22.csv")
